In [1]:
import torch
if torch.cuda.is_available():
  device = torch.device("cuda")
  print('There are %d GPU(s) available.' % torch.cuda.device_count())
  print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")


There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [2]:
!pip install transformers

     |████████████████████████████████| 2.5MB 8.1MB/s 
     |████████████████████████████████| 901kB 50.6MB/s 
     |████████████████████████████████| 3.3MB 45.7MB/s 


In [3]:

!pip install wget

  Created wheel for wget: filename=wget-3.2-cp37-none-any.whl size=9675 sha256=e6fe8dd0b5c558c116ae20cc114fdbe1555b4a559d19c7f642f57caf045bff1a
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [4]:

import os
from google.colab import files
uploaded = files.upload()


Saving Target Identification Task .csv to Target Identification Task .csv


In [5]:
import pandas as pd
df = pd.read_csv('Target Identification Task .csv')
print('Number of training sentences: {:,}\n'.format(df.shape[0]))
df.sample(10)


Number of training sentences: 4,935



,ID,Text,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Label
3164,3165,i have no will of my own.,NaN,NaN,NaN,NaN,TARGETED
4452,4453,"i never went out, i never went with anyone else.",NaN,NaN,NaN,NaN,TARGETED
792,793,"And I was alone all the while, despite the fac...",NaN,NaN,NaN,NaN,TARGETED
2253,2254,"I was promised NAME and instead I got NAME, Be...",NaN,NaN,NaN,NaN,TARGETED
2136,2137,"As I hustled year in and year out, fought thro...",NaN,NaN,NaN,NaN,TARGETED
4143,4144,i can never recover from my childhood loneliness.,NaN,NaN,NaN,NaN,TARGETED
3451,3452,"they should all be in prison, they are horribl...",NaN,NaN,NaN,NaN,TARGETED
2337,2338,YOU NEED TO BE STRONG FOR MOM.,NaN,NaN,NaN,NaN,UNTARGETED
262,263,The teachers didn?t do ANYTHING!,NaN,NaN,NaN,NaN,UNTARGETED
4597,4598,"you have been thru ""hell"" with me since then.",NaN,NaN,NaN,NaN,TARGETED


In [6]:
possible_labels = df.Label.unique()
label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

{'TARGETED': 0, 'UNTARGETED': 1}

In [7]:
print(df.groupby("Label").count())

              ID  Text  Unnamed: 2  Unnamed: 3  Unnamed: 4  Unnamed: 5
Label                                                                 
TARGETED    3608  3608           0           0           0           0
UNTARGETED  1327  1327           0           0           0           0


In [8]:
df['Label'] = df.Label.replace(label_dict)

In [9]:
df.loc[df.Label == 0].sample(5)[['Text', 'Label']]

,Text,Label
4627,i think that if someone who knows me reads thi...,0
3406,i hope you will soon get a kind and more prosp...,0
4815,i just wanted to find a place where i could be...,0
2720,driving along check tile abyss at my left like...,0
2243,Whatever the lawyers decide to do I don't care.,0


In [10]:
sentences = df.Text.values
labels = df.Label.values

In [11]:
from transformers import BertTokenizer
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Loading BERT tokenizer...


In [12]:
print(' Original: ', sentences[0])
print('Tokenized: ', tokenizer.tokenize(sentences[0]))
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentences[0])))

 Original:  I am going to commit suicide and nobody is responsible for my death
Tokenized:  ['i', 'am', 'going', 'to', 'commit', 'suicide', 'and', 'nobody', 'is', 'responsible', 'for', 'my', 'death']
Token IDs:  [1045, 2572, 2183, 2000, 10797, 5920, 1998, 6343, 2003, 3625, 2005, 2026, 2331]


In [13]:
max_len = 0
for sent in sentences:
  input_ids = tokenizer.encode(sent, add_special_tokens=True)
  max_len = max(max_len, len(input_ids))
print('Max sentence length: ', max_len)

Max sentence length:  157


In [14]:
input_ids = []
attention_masks = []

# For every sentence...
for sent in sentences:
   
    encoded_dict = tokenizer.encode_plus(
                        sent,                     
                        add_special_tokens = True, 
                        max_length = 256,           
                        pad_to_max_length = True,
                        return_attention_mask = True,   
                        return_tensors = 'pt',   
                   )
    
  
    input_ids.append(encoded_dict['input_ids'])
    
   
    attention_masks.append(encoded_dict['attention_mask'])

input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

# Print sentence 0, now as a list of IDs.
print('Original: ', sentences[0])
print(labels[0])
print('Token IDs:', input_ids[0])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2132: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Original:  I am going to commit suicide and nobody is responsible for my death
tensor(0)
Token IDs: tensor([  101,  1045,  2572,  2183,  2000, 10797,  5920,  1998,  6343,  2003,
         3625,  2005,  2026,  2331,   102,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,

In [15]:
from torch.utils.data import TensorDataset, random_split
dataset = TensorDataset(input_ids, attention_masks, labels)
train_size = int(0.9 * len(dataset))

val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))


4,441 training samples
  494 validation samples


In [16]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
batch_size = 32
train_dataloader = DataLoader(
            train_dataset,  
            sampler = RandomSampler(train_dataset), 
            batch_size = batch_size 
        )
validation_dataloader = DataLoader(
            val_dataset, 
            sampler = SequentialSampler(val_dataset), 
            batch_size = batch_size 
        )

In [17]:
from transformers import BertForSequenceClassification, AdamW, BertConfig
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels = 2,
    output_attentions = False,
    output_hidden_states = False,
  )
model.cuda()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [18]:
params = list(model.named_parameters())
print('The BERT model has {:} different named parameters.\n'.format(len(params)))
print('==== Embedding Layer ====\n')
for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))
print('\n==== First Transformer ====\n')
for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))
print('\n==== Output Layer ====\n')
for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (30522, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (

In [19]:
optimizer = AdamW(model.parameters(),
                   lr = 2e-5,
                   eps = 1e-8
            )

In [20]:
from transformers import get_linear_schedule_with_warmup
epochs = 4
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, 
                                            num_training_steps = total_steps)

In [21]:
import numpy as np
def flat_accuracy(preds, labels):
   pred_flat = np.argmax(preds, axis=1).flatten()
   labels_flat = labels.flatten()
   return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [22]:
import time
import datetime
def format_time(elapsed):
  elapsed_rounded = int(round((elapsed)))
  return str(datetime.timedelta(seconds=elapsed_rounded))


In [23]:
import random
import numpy as np

seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
training_stats = []
total_t0 = time.time()

for epoch_i in range(0, epochs):
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')
    t0 = time.time()
    total_train_loss = 0
    model.train()
    for step, batch in enumerate(train_dataloader):
      if step % 40 == 0 and not step == 0:
         elapsed = format_time(time.time() - t0)
         print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

      b_input_ids = batch[0].to(device)
      b_input_mask = batch[1].to(device)
      b_labels = batch[2].to(device)

      model.zero_grad()  

      output = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)

      loss=output.loss
      logits=output.logits
    
      total_train_loss +=loss.item()

      loss.backward()

      torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

      optimizer.step()

      scheduler.step()


    avg_train_loss = total_train_loss / len(train_dataloader) 
    training_time = format_time(time.time() - t0)
    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))

    print("")
    print("Running Validation...")

    t0 = time.time()
    model.eval()

    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    for batch in validation_dataloader:
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        with torch.no_grad(): 
          output = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)

        loss=output.loss
        logits=output.logits
    
        total_train_loss +=loss.item()
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        total_eval_accuracy += flat_accuracy(logits, label_ids)
      
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    validation_time = format_time(time.time() - t0)
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
     )
print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


    
        
 

  



======== Epoch 1 / 4 ========
Training...
  Batch    40  of    139.    Elapsed: 0:00:51.
  Batch    80  of    139.    Elapsed: 0:01:44.
  Batch   120  of    139.    Elapsed: 0:02:39.

  Average training loss: 0.18
  Training epcoh took: 0:03:05

Running Validation...
  Accuracy: 0.98
  Validation Loss: 0.00
  Validation took: 0:00:08

======== Epoch 2 / 4 ========
Training...
  Batch    40  of    139.    Elapsed: 0:00:59.
  Batch    80  of    139.    Elapsed: 0:01:56.
  Batch   120  of    139.    Elapsed: 0:02:54.

  Average training loss: 0.05
  Training epcoh took: 0:03:22

Running Validation...
  Accuracy: 0.98
  Validation Loss: 0.00
  Validation took: 0:00:09

======== Epoch 3 / 4 ========
Training...
  Batch    40  of    139.    Elapsed: 0:00:58.
  Batch    80  of    139.    Elapsed: 0:01:56.
  Batch   120  of    139.    Elapsed: 0:02:54.

  Average training loss: 0.04
  Training epcoh took: 0:03:21

Running Validation...
  Accuracy: 0.98
  Validation Loss: 0.00
  Validation too

In [24]:
import pandas as pd
pd.set_option('precision', 2)
df_stats = pd.DataFrame(data=training_stats)
df_stats = df_stats.set_index('epoch')
df_stats

,Training Loss,Valid. Loss,Valid. Accur.,Training Time,Validation Time
epoch,,,,,
1,0.18,0.0,0.98,0:03:05,0:00:08
2,0.05,0.0,0.98,0:03:22,0:00:09
3,0.04,0.0,0.98,0:03:21,0:00:09
4,0.02,0.0,0.98,0:03:21,0:00:09


In [31]:
import pandas as pd
df = pd.read_csv('Target Identification Task .csv')


print('Number of test sentences: {:,}\n'.format(df.shape[0]))

sentences = df.Text.values


input_ids = []
attention_masks = []

for sent in sentences:
  encoded_dict = tokenizer.encode_plus(
                        sent,                     
                        add_special_tokens = True, 
                        max_length = 64,           
                        pad_to_max_length = True,
                        return_attention_mask = True,   
                        return_tensors = 'pt',   
                   )
    
  
  input_ids.append(encoded_dict['input_ids'])
    
   
  attention_masks.append(encoded_dict['attention_mask'])


  

input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)


batch_size = 32  

prediction_data = TensorDataset(input_ids, attention_masks, labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)



Number of test sentences: 4,935



/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2132: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


In [32]:
print('Predicting labels for {:,} test sentences...'.format(len(input_ids)))
model.eval()
predictions , true_labels = [], []

for batch in prediction_dataloader:
  batch = tuple(t.to(device) for t in batch)
  b_input_ids, b_input_mask, b_labels = batch
  with torch.no_grad():
    outputs = model(b_input_ids, token_type_ids=None, 
                      attention_mask=b_input_mask)
  logits = outputs[0]
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()

  predictions.append(logits)
  true_labels.append(label_ids)

print('    DONE.')

Predicting labels for 4,935 test sentences...
    DONE.


In [42]:
from sklearn.metrics import matthews_corrcoef

matthews_set = []
print('Calculating Matthews Corr. Coef. for each batch...')
for i in range(len(true_labels)):
   pred_labels_i = np.argmax(predictions[i], axis=1).flatten()
   matthews = matthews_corrcoef(true_labels[i], pred_labels_i)                
   matthews_set.append(matthews)

Calculating Matthews Corr. Coef. for each batch...


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


In [44]:
flat_predictions = np.concatenate(predictions, axis=0)


flat_predictions = np.argmax(flat_predictions, axis=1).flatten()


flat_true_labels = np.concatenate(true_labels, axis=0)


mcc = matthews_corrcoef(flat_true_labels, flat_predictions)

print('Total MCC: %.3f' % mcc)

Total MCC: 0.984
